In [1]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_gigachat.chat_models import GigaChat
from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState, START
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict

import datetime
import warnings
import os
import re

In [22]:
!pip list | grep gigachat

gigachat                  0.1.39.post1
langchain-gigachat        0.3.10


In [2]:
warnings.filterwarnings("ignore")

In [3]:
os.environ["GIGACHAT_CREDENTIALS"] = "Yjc1YWZhNTItMzYwYS00NmU4LTk4YjctZjU4YzAwMDIyMGJmOjJhNWE3YmVhLTAyYzctNGJhNy05NWE3LWEzY2YwNGQzYzZiNw=="

In [4]:
with open("../sources/ba_instruction.md", "r") as f:
    ba_instruction = f.read()

def save_result(result):
    format_ = "%Y_%m_%dT%H_%M_%S"
    datetime_mark = datetime.datetime.now().strftime(format_)
    
    with open(f"../tmp/ba_{datetime_mark}.md", "w") as f:
        f.write(result["result"])

In [5]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False,
    streaming=False,
    max_tokens=8192,
    temperature=0.3,
    repetition_penalty=1.01,
    timeout=180
)

In [6]:
tools = []

In [7]:
ba_agent = create_react_agent(llm, tools=[], checkpointer=MemorySaver())

In [8]:
class AgentState(TypedDict):
    task: str
    messages: List
    result: str
    questions: List
    
class ValidatorResponse(TypedDict):
    """В поле instructions нужно указать рекомендации. В поле status указать статус задачи [FAIL, SUCCESS]"""
    instructions: str
    status: Literal["FAIL", "SUCCESS"]

In [9]:
def user_node(state:AgentState)->Command[Literal["ba_agent"]]:    
    print("Status: user_node")
    print(state["questions"])
    msg = input()
    if "messages" in state and state["messages"]:
        old_messages = state["messages"]
    else:
        old_messages = []
    return Command(
        update={
            "messages": old_messages + [HumanMessage(content=msg)]
        },
        goto="ba_agent"
    )

In [10]:
def ba_agent_node(state:AgentState)->Command[Literal["ba_validator", "user"]]:    
    print("Status: ba_agent_node")
    if "messages" in state and state["messages"]:
        old_messages = state["messages"]
        request = state["messages"][-1].content
    else:
        old_messages = []
        request = ba_prompt.format(task=state["task"], ba_instruction=ba_instruction)
    request = {
            "messages": [HumanMessage(content=request)]
        }
    response = ba_agent.invoke(request, config=config)
    if isinstance(response, dict):
        result = response["messages"][-1].content
    else:
        result = response
    matches = re.findall(r'\[ВОПРОС\](.*?)\[/ВОПРОС\]', result, re.DOTALL)
    if matches:
        return Command(
            update={
                "questions": matches
            },
            goto="user"
        )
        
    return Command(
        update={
            "result": result,
            "messages": old_messages + [HumanMessage(content=result, name="Аналитик")]
        },
        goto="ba_validator"
    )

def ba_validator_node(state:AgentState)->Command[Literal["ba_agent", END]]:
    print("Status: ba_validator_node")
    return_node = "ba_agent"
    if not "messages" in state or not state["messages"]:
        return Command(goto=return_node)
    
    prompt = ba_validator_prompt.format(task=state["task"], format=format_)
    system = SystemMessage(content=prompt)
    request = [system] + state["messages"]

    validator_msgs_fount = len(list(filter(lambda x: x.name=="Валидатор", state["messages"])))
    response = llm.invoke(request)
    
    response = response.content
    print(f"RESPONSE: {response}")
    match_json = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if match_json:
        response = eval(match_json.group(1))
    else:
        response = {"status":"SUCCESS", "instructions":""}
    
    result = response["instructions"]
    print(f"STATUS: {response['status']}\n\nVALIDATOR: {result}")
    
    if response["status"]=="SUCCESS" or validator_msgs_fount+1 > 3:
        goto = END
    else:
        goto = return_node
    return Command(
            update={
            "messages": state["messages"] + [HumanMessage(content=result, name="Валидатор")],
            },
            goto=goto
        )

In [27]:
ba_prompt = (
    "Ты - превосходный бизнес-аналитик."
    " Твоя задача - составить бизнес-требования к задаче {task} в соответствии с инструкцией {ba_instruction}."
    " Если для составления требований тебе необходима дополнительная информация, сформируй вопросы и оберни каждый вопрос в теги [ВОПРОС][/ВОПРОС]."
    "\n\nПример: [ВОПРОС]Кто основные потребители сервиса[/ВОПРОС]"
    "\n[ВОПРОС]Какие сроки выполнения проекта?[/ВОПРОС]"
)

In [28]:
print(ba_prompt)

Ты - превосходный бизнес-аналитик. Твоя задача - составить бизнес-требования к задаче {task} в соответствии с инструкцией {ba_instruction}. Если для составления требований тебе необходима дополнительная информация, сформируй вопросы и оберни каждый вопрос в теги [ВОПРОС][/ВОПРОС].

Пример: [ВОПРОС]Кто основные потребители сервиса[/ВОПРОС]
[ВОПРОС]Какие сроки выполнения проекта?[/ВОПРОС]


In [12]:
format_ ="""{
    "instructions": "Здесь укажи свои рекомендации или оставь строку пустой, если рекомендаций нет",
    "status": "FAIL" если требуется доработка иначе "SUCCESS"
}"""

In [13]:
ba_validator_prompt = """Ты - Валидатор. Ты прекрасно умеешь критиковать написанные бизнес-требования для достижения наилучшего результата. 
Ты работаешь вместе с Аналитиком. Проанализуруй бизнес-требования, которые предложил аналитик к задаче {task}. 
- Проверь, что бизнес-требования, которые предложил аналитик, хорошо подходят для решения задачи.
- Проверь, что хорошо описаны как основные сценарии, так и альтернативные.
- Проверь, что хорошо явно выделена User Story.

Если сценарий нужно улучшать, напиши свои замечания и верни сценарий на дорабоку, ответив FAIL В конце инструкции попроси аналитика заново полностью описать требования, а не вносить изменения в существующие. 
Если Аналитик хорошо выполнил свою работу и замечаний нет, ответь SUCCESS.

Ответ дай в следующем формате json:
```json
{format}
```
"""

In [14]:
config = {"configurable": {"thread_id": "thread_id12345"}}

In [15]:
state_schema = AgentState
builder = StateGraph(state_schema=state_schema)
builder.set_entry_point("ba_agent")
builder.add_node("user", user_node)
builder.add_node("ba_agent", ba_agent_node)
builder.add_node("ba_validator", ba_validator_node)
graph = builder.compile()

In [16]:
with open("../tmp/desc_2025_05_18T03_55_01.md", "r") as f:
    task = f.read() 

In [17]:
initial_state = {
    "task": task
}

In [21]:
print(result["result"])

### Бизнес-требования к задаче автоматизации обработки заявок

#### 1. Резюме
В компании существует проблема неэффективной обработки заявок на инциденты, что приводит к увеличению времени реакции на инциденты и снижению удовлетворенности клиентов. Для решения этой проблемы предлагается автоматизировать процесс обработки заявок с помощью системы, состоящей из пяти агентов: анализатора, сравнивателя, заполнителя, валидатора и оркестратора. Автоматизация позволит сократить время обработки заявок, повысить точность и снизить нагрузку на сотрудников.

#### 2. Предпосылки и необходимость
В текущей системе обработка заявок осуществляется вручную, что приводит к следующим проблемам:
- Длительное время обработки заявок.
- Высокий уровень ошибок при заполнении заявок.
- Низкая удовлетворенность клиентов.
- Высокая нагрузка на сотрудников.

Для устранения этих проблем необходимо автоматизировать процесс обработки заявок, что позволит:
- Сократить время обработки заявок.
- Повысить точность заполн

In [18]:
result = graph.invoke(initial_state)

Status: ba_agent_node
Status: user_node
['Какие проблемы в текущей системе обработки заявок приводят к необходимости автоматизации?', 'Какие конкретные показатели эффективности (KPI) планируется улучшить с помощью автоматизации?', 'Какие технические ограничения существуют в текущей инфраструктуре OpenShift и базе данных "ПКАП"?', 'Какие дополнительные инструменты или технологии планируется использовать для реализации проекта?', 'Какие риски могут возникнуть при реализации проекта и как они будут минимизированы?', 'Какие ресурсы (человеческие, финансовые, технические) необходимы для реализации проекта?', 'Какие сроки выполнения проекта и его этапов?', 'Какие ожидаемые затраты и доходы от реализации проекта?']


 Придумай сам


Status: ba_agent_node
Status: ba_validator_node
RESPONSE: ```json
{
    "instructions": "В предоставленных бизнес-требованиях отсутствует описание конкретных сценариев использования (User Stories), что затрудняет понимание того, как именно система будет использоваться конечными пользователями. Также не указаны конкретные критерии оценки успешности проекта, такие как метрики для измерения достижения поставленных целей. Необходимо добавить раздел с описанием сценариев использования и критериев оценки успешности проекта.",
    "status": "FAIL"
}
``` 

Пожалуйста, полностью опиши заново бизнес-требования с учетом указанных замечаний.
STATUS: FAIL

VALIDATOR: В предоставленных бизнес-требованиях отсутствует описание конкретных сценариев использования (User Stories), что затрудняет понимание того, как именно система будет использоваться конечными пользователями. Также не указаны конкретные критерии оценки успешности проекта, такие как метрики для измерения достижения поставленных целей. Необ

In [60]:
save_result(result)